## Creating the eci File
Now that we have run the vasp calculations, we can generate an eci file for those structures. Then we can produce the eci vs cluster diameter plot. 



In [ ]:
def eciCalculations(file_suffix, db_name):
    ## begin import block
    from clease import Concentration
    from clease import CEBulk
    from ase.db import connect
    from clease import Evaluate
    ## end import block
    
    
    ## begin block- creates ECI file 
    conc=Concentration(basis_elements=[['Au', 'Cu']])
    conc.set_conc_formula_unit(formulas=["Au<x>Cu<1-x>"],
                               variable_range={"x": (0, 1.0)})
    settings=CEBulk(crystalstructure='fcc', a=3.8, size=[3,3,3], 
                    concentration=conc, db_name=db_name,
                    max_cluster_size=4, 
                    max_cluster_dia=[6.0,5.0,5.0])
    eva= Evaluate(setting=setting, scoring_scheme='k-fold', nsplits=10)
    eva.set_fitting_scheme(fitting_scheme='l1')
    alpha= eva.plot_CV(alpha_min=1E-8, alpha_max=10, num_alpha=1000)
    
    eva.set_fitting_scheme(fitting_scheme='l1', alpha=alpha)
    eva.plot_fit()
    eva.plot_ECI()
    eva.save_eci(fname='eci_l1_VASP_%s'%file_suffix)
    
    with open(file_name) as json_file:
        eci=json.load(json_file)
    print("Writing default structure")
    default_structure=settings.prim_no_bkg()*[3,3,3]
    default_structure.write('au1000.vasp')
    
    distance_matrix=eva._distance_from_names()
    eva_matrix=list(eva.eci)
    length=len(distance_matrix)
    three_bodies=[]
    two_bodies=[]
    four_bodies=[]
    three_bodies_energies=[]
    two_bodies_energies=[]
    four_bodies_energies=[]


    for i in range(length):
        string2read=eva.cf_names[i]
        body=string2read[1]   
        bodyint=int(body)
        energy_value=eva_matrix[i]
        if distance_matrix[i]<=4 and bodyint>=2 and energy_value!=0:
            #print(bodyint)
            if bodyint ==2:
                #print("2 Body Cluster, separating")
                #print()
                two_bodies.append(distance_matrix[i])
                two_bodies_energies.append(energy_value)
            elif bodyint ==3:
                #print("3 Body Cluster, separating")
                #print()
                three_bodies.append(distance_matrix[i]+.33)
                three_bodies_energies.append(energy_value)
            else:
                #print("4 Body Cluster, separating")
                #print()
                four_bodies.append(distance_matrix[i]-0.33)
                four_bodies_energies.append(energy_value)
        else:
            print('passing')
            print()
    print()


    print(two_bodies, three_bodies, four_bodies)
    print(two_bodies_energies, three_bodies_energies, four_bodies_energies)
    
    
    
    import matplotlib.pyplot as plt
    import numpy as np
    number_groups=3

    fig, ax = plt.subplots()
    bar_width = 0.35
    opacity = 0.8
    gap=.5

    labels = ['G1', 'G2', 'G3', 'G4', 'G5']
    x=np.arange(len(labels))
    two_body = ax.bar(two_bodies,two_bodies_energies , bar_width,
                      alpha=opacity, color='b', label='2Body')

    three_body = ax.bar(three_bodies, three_bodies_energies, bar_width,
                        alpha=opacity, color='r', label='3Body')

    four_body=plt.bar(four_bodies, four_bodies_energies, bar_width,
                      alpha=opacity, color='g', label='4Body')
    ax.axhline(0.0, ls='-', color="black", lw=0.5)
    plt.xlabel("Cluster diameter ($n^{th}$ nearest neighbor)")
    plt.ylabel("ECI (eV/atom)")
    plt.legend(loc='best')
    plt.show()